In [3]:
%load_ext autoreload
%autoreload 2


In [4]:
import utils 
import numpy as np
import pandas as pd
import os
import sys
import numpy as np
import tkinter
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate
from sklearn.metrics import confusion_matrix
import pickle
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [5]:
pd.set_option('display.max_colwidth', -1)
root_dir = os.path.abspath('.')
data_dir = os.path.join(root_dir, 'dataset')
train = pd.read_csv(os.path.join(data_dir,'train.csv'))
test = pd.read_csv(os.path.join(data_dir, 'test.csv'))

test_x, _ = utils.featurize(test, stage='test')


train_x, train_y = utils.featurize(train)
print(test_x[0:10])

[[0 0 0 1 0 0 0 4 0 0 0 0 0 0 1 0 0 0 2 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 5 0 0 0 0 0 5 0 5 5 5 5 5 5]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 6 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 2 0 0 0 0 0 2 1 2 2 2 2 2 2]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 5 0 0 0 0 0 5 5 5 5 5 5 5 5]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 3 0 0 0 0 0 0]]


In [4]:
def get_proba(clf,x):
    a = []
    for v in clf.predict_proba(x.reshape(1,-1)):
        a.append([v[0,1]])
       
    return a



In [5]:
tree = DecisionTreeClassifier(criterion="entropy",max_depth=3)
cv_results = cross_validate(tree,train_x,train_y,cv=5,return_estimator=True,return_train_score=False)

In [7]:
def column(matrix, i):
    return [row[i] for row in matrix]

In [7]:
print(cv_results['test_score'])
clf=cv_results["estimator"][0]

# print(np.shape(clf.predict_proba(train_x[6].reshape(1, -1))))
# clf.predict_proba(train_x[6].reshape(1, -1))[3][0][1]
f = open("decision_tree.csv","w")
k = 0
for feat in test_x:
    prob = get_proba(clf,feat)
    f.write(str(test.values[k][0]) + "," +",".join(str(x[0]) for x in prob))
    f.write("\n")
    k+=1
f.close()

[0.90202099 0.90524535 0.90646738 0.90477533 0.90606004]


In [8]:
rf = RandomForestClassifier(n_estimators=100)
rf_cv_results=cross_validate(rf,train_x,train_y,cv=5,return_estimator=True,return_train_score=False)
print(rf_cv_results['test_score'])

[0.90484098 0.90631071 0.90928746 0.90743874 0.90750141]


In [11]:
rf_clf = rf_cv_results['estimator'][0]
k=0
ran_f = open("random_forest.csv","w")
for feat in test_x:
    prob = get_proba(rf_clf,feat)
    ran_f.write(str(test.values[k][0]) + "," +",".join(str(x[0]) for x in prob))
    ran_f.write("\n")
    k+=1
ran_f.close()

In [14]:
lr = []
lr.append(LogisticRegression())
lr.append(LogisticRegression())
lr.append(LogisticRegression())
lr.append(LogisticRegression())
lr.append(LogisticRegression())
lr.append(LogisticRegression())
lr_cv_results = []
i=0
while(i<6):
    
    lr_cv_result = cross_validate(lr[i],train_x,column(train_y,i),cv=5,return_estimator=True,return_train_score=False)
    print(lr_cv_result['test_score'])
    lr_cv_results.append(lr_cv_result)
    i+=1

[0.93990287 0.94018487 0.94077834 0.93996365 0.9398991 ]
[0.99006737 0.99050573 0.98972238 0.99016106 0.99041173]
[0.96371612 0.96343412 0.96374632 0.96506236 0.96289913]
[0.99696068 0.99696068 0.99689801 0.99699182 0.99699182]
[0.95484882 0.95710481 0.95757348 0.95431472 0.95456397]
[0.99128936 0.99122642 0.99113242 0.99106975 0.99122642]


In [8]:
def lr_get_proba(clf,x):
    a = []
    for i in clf:
        
        for v in i.predict_proba(x.reshape(1,-1)):
            a.append(v[1])
       
    return a



In [38]:
lr_clfs = []
lr_clfs.append(lr_cv_results[0]['estimator'][0])
lr_clfs.append(lr_cv_results[1]['estimator'][0])
lr_clfs.append(lr_cv_results[2]['estimator'][0])
lr_clfs.append(lr_cv_results[3]['estimator'][0])
lr_clfs.append(lr_cv_results[4]['estimator'][0])
lr_clfs.append(lr_cv_results[5]['estimator'][0])

",".join(str(x) for x in lr_get_proba(lr_clfs,test_x[1]))

'0.06196153295436414,0.010201813287534878,0.03273787179065023,0.004423638436312418,0.04763677331523011,0.01071707247351741'

In [39]:
k=0
lr_f = open("logistic_regression.csv","w")
for feat in test_x:
    prob = lr_get_proba(lr_clfs,feat)
    lr_f.write(str(test.values[k][0]) + "," +",".join(str(x) for x in prob))
    lr_f.write("\n")
    k+=1
lr_f.close()

In [16]:
svc = []
svc.append(svm.SVC(probability=True))
svc.append(svm.SVC(probability=True))
svc.append(svm.SVC(probability=True))
svc.append(svm.SVC(probability=True))
svc.append(svm.SVC(probability=True))
svc.append(svm.SVC(probability=True))
svc_cv_results = []
j=0
while(j<6):
    svc_cv_result = cross_validate(svc[j],train_x,column(train_y,j),cv=3,return_estimator=True,return_train_score=False)
    print(svc_cv_result['test_score'])
    svc_cv_results.append(svc_cv_result)
    j+=1

[0.94113666 0.94164317 0.94070314]
[0.99003591 0.99005471 0.99007314]
[0.97283375 0.97407407 0.97382967]
[0.99699197 0.99701072 0.99701072]
[0.96350886 0.96474968 0.96356389]
[0.99133312 0.99133296 0.99142696]


In [17]:
svc_clfs = []
svc_clfs.append(svc_cv_results[0]['estimator'][0])
svc_clfs.append(svc_cv_results[1]['estimator'][0])
svc_clfs.append(svc_cv_results[2]['estimator'][0])
svc_clfs.append(svc_cv_results[3]['estimator'][0])
svc_clfs.append(svc_cv_results[4]['estimator'][0])
svc_clfs.append(svc_cv_results[5]['estimator'][0])

count = 0
for classifier in svc_clfs:
    with open("svc"+str(count)+".pickle","wb") as f:
        pickle.dump(classifier,f)
    count+=1
",".join(str(x) for x in lr_get_proba(svc_clfs,test_x[1]))

'0.054940068937689636,0.009839087747827628,0.019901577256955728,0.0031495311437181855,0.02683736324712405,0.008556809949554831'

In [18]:
k=0
lr_f = open("support_vector_machine.csv","w")
for feat in test_x:
    prob = lr_get_proba(svc_clfs,feat)
    lr_f.write(str(test.values[k][0]) + "," +",".join(str(x) for x in prob))
    lr_f.write("\n")
    k+=1
lr_f.close()

In [10]:
# when ready, set this to the best model you found, trained on the test data:
best_classifier = None

with open('classifier.pickle', 'wb') as f: # 'wb' stands for 'write bytes'
    pickle.dump(best_classifier, f)